In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import sys
import gym

# 簡化版的deep q learning： 僅把q-table換成類神經網路

在 Reinforcement Learning 中，我們都利用 $Q_{target}$（把它當作真實的Q值) 來更新神經網路的 weights。

公式：

$$ Q_{target} = Q(s) + \alpha \cdot (R(s, a) + Q(s_{next}) \cdot \gamma - Q(s)) $$

($s_{next}$ 代表下一步的狀態，下一步的狀態有很多種可能，我們這裡選擇的 $s_{next}$ 是能得到最大Q的狀態，這種方法是比較 aggressive 的方法，還有另外一種是SARSA有興趣可以自尋搜尋一下；$\alpha$ 這邊我們設定為1）

因此公式就變成 

$$ Q_{target} = R(s, a) + max(Q(s_{next}, a)) \cdot \gamma $$  

In [3]:
class QLearning:
    
    def __init__(
        self, 
        n_actions,             # 動作的維度，例如上下左右就有四維
        n_states,              # 用來描述狀態的維度，例如馬力歐在平面上就是二維
        gamma = 0.9,           # 遠見程度，值越大越以未來收益為重
        epsilon = 0.9,         # 保守程度，越大就越容易用Q值大小來採取行動；越小則越容易產生隨機行動
        learning_rate = 0.001  # 神經網路的更新率
    ):
        # 變數給初始值
        self.n_actions = n_actions
        self.n_states = n_states
        self.gamma = gamma
        self.epsilon = epsilon
        self.lr = learning_rate
        
        tf.reset_default_graph() ## 重新 build graph 需要跑這行
        self.sess = tf.Session() ## 宣告session
        
        # 輸入 current state
        self.state_input = tf.placeholder(shape=[None, self.n_states], name='input', dtype=tf.float32)
        
        # q_target = R(s, action) + Q(s_)*Gamma 
        self.q_target = tf.placeholder(shape=[None, self.n_actions], name='q_target', dtype=tf.float32)
        
        # 搭建神經網路
        with tf.variable_scope('Q_table'):
            self.q_eval = self.build_network('net_eval') 
        
        # 管理神經網路的 parameters
        self.Qnet_eval_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Q_table/net_eval')
        
        # 計算 q_target 和 q_eval 的 MSE 來更新神經網路的參數
        self.loss = tf.reduce_mean(tf.squared_difference(self.q_target, self.q_eval))
        self.train = tf.train.RMSPropOptimizer(self.lr).minimize(self.loss, var_list=self.Qnet_eval_params)
        
        # 將神經網路初始化
        self.sess.run(tf.global_variables_initializer()) 
            
    def build_network(self, scope): 
        with tf.variable_scope(scope):
            x_h1 = tf.layers.dense(inputs=self.state_input, units=5, activation=tf.nn.tanh)
            x_h2 = tf.layers.dense(inputs=x_h1, units=5, activation=tf.nn.tanh)
            
        # 輸出 "不同動作" 對應的Q值
        return tf.layers.dense(inputs=x_h2, units=self.n_actions)
            
    def choose_action(self, current_state):
        """
        利用 epsilon 來控制探索的隨機程度，通常探索初期會給比較小的 epsilon 增加行為的隨機程度，
        然後隨著遊戲的進行慢慢增加 epsilon。不過由於這裡的遊戲較簡單，就不做此設定。
        """
        if np.random.uniform() < self.epsilon: 
            # 選擇產生估計Q值較大的行動
            q_eval = self.sess.run(self.q_eval, feed_dict={self.state_input: current_state[np.newaxis, :]})
            self.action = np.argmax(q_eval)
        else:
            # 採取隨機行動
            self.action = np.random.randint(0, self.n_actions)

        return self.action
    
    def learn(self, current_state, reward, next_state):
        # 算出實際 Q 值並用此更新神經網路參數
        q_eval = self.sess.run(self.q_eval, feed_dict={self.state_input: current_state[np.newaxis, :]})
        q_eval_next = self.sess.run(self.q_eval, feed_dict={self.state_input: next_state[np.newaxis, :]})
        q_target = q_eval.copy()
        q_target[:, self.action] = reward + self.gamma*q_eval_next.max()
        _, self.cost = self.sess.run(
            [self.train, self.loss],
            feed_dict={self.state_input: current_state[np.newaxis, :], self.q_target: q_target})

    def model_save(self, model_name):        
        saver = tf.train.Saver()
        saver.save(self.sess, "saved_models/{}.ckpt".format(model_name))
    
    def model_restore(self, model_name):
        saver = tf.train.Saver()
        saver.restore(self.sess, "saved_models/{}.ckpt".format(model_name))

# 讓RL Agent開始與環境互動吧～

**提示**

在/Users/Yourname/anaconda3/lib/python3.6/site-packages/gym/envs底下可以找到Gym AI底下所有遊戲的文件，其中__init__.py定義了呼叫各個遊戲的名稱，例如moutain car你就得用gym.make(‘MountainCar-v0’)，另外和遊戲相關的py檔在envs/classic_control的資料夾內。我們接下來要玩的是離散版本的不是連續版的喔～，另外如果您找不到的話我們也將檔案拉出來放在gym document供大家參考。

In [6]:
env = gym.make('MountainCar-v0')
"""
執行gym ai的遊戲時也請加下面這兩行
"""
env = env.unwrapped
env.seed(1)

RL = QLearning(n_actions = 3, 
               n_states = 2,
               gamma = 0.9,
               epsilon = 0.8,
               learning_rate = 0.01
               )

reward_result = []

# 初始化環境並取得起始狀態
total_reward = 0
current_state = env.reset()

while True:
    # 產生環境視窗
    env.render()

    # 依據目前狀態，選擇下一步行動
    action = RL.choose_action(current_state)
    
    """
    Gym ai 的遊戲step都會output 4個值，分別為下一狀態、
    獎勵、回合結束與否和info，不過info我們用不到因此不用管它
    """
    
    # 採取行動，從環境中取得更新狀態、獎勵、遊戲是否結束
    next_state, reward, done, _ = env.step(action)

    total_reward+= reward

    RL.learn(current_state, reward, next_state)

    # break while loop when end of this episode
    if done:
        print('Total Reward:{}'.format(total_reward))
        reward_result.append(total_reward)
        break
    # swap state
    current_state = next_state  
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Total Reward:-6812.0
